In [1]:
# Lista de templates a remover
lista_a_remover = ["tput no value for term and no t specified",
                    "find no such file or directory",
                    "mv cannot stat no such file or directory",
                    "sed can t read temp no such file or directory",
                    "compile failed using"
                ]

# Arquivo de entrada e saída
arquivo_entrada = "templates_filtrados.log"
arquivo_saida = "templates_filtrados_v2.log"

with open(arquivo_entrada, 'r', encoding='utf-8') as f_in, open(arquivo_saida, 'w', encoding='utf-8') as f_out:
    for linha in f_in:
        # Divide a linha por vírgulas e remove espaços extras
        partes = [p.strip() for p in linha.strip().split(',')]

        # Filtra os templates que não estão na lista a remover
        partes_filtradas = [p for p in partes if p not in lista_a_remover]

        # Escreve no novo arquivo
        nova_linha = ', '.join(partes_filtradas)
        f_out.write(nova_linha + '\n')


In [2]:
# Arquivo de entrada e saída
arquivo_entrada = "templates_filtrados_v2.log"
arquivo_saida = "templates_filtrados_v3.log"

with open(arquivo_entrada, 'r', encoding='utf-8') as f_in, open(arquivo_saida, 'w', encoding='utf-8') as f_out:
    for linha in f_in:
        # Divide por vírgulas e remove espaços em branco
        partes = [p.strip() for p in linha.strip().split(',') if p.strip() != '']

        # Escreve apenas se houver mais de um template
        if len(partes) > 1:
            f_out.write(linha)


In [3]:
# --- Célula 1: imports e checagem de ijson (streaming para JSONs grandes)
import json
from pathlib import Path

try:
    import ijson  # opcional: pip install ijson
    HAS_IJSON = True
except Exception:
    HAS_IJSON = False

# --- Célula 2: funções utilitárias

def _walk(obj):
    """Percorre recursivamente o objeto e produz cada valor de 'template' encontrado."""
    if isinstance(obj, dict):
        # caminho típico: ... -> 'group' (list) -> item(dict) -> 'template'
        grp = obj.get('group')
        if isinstance(grp, list):
            for item in grp:
                if isinstance(item, dict) and 'template' in item:
                    yield item['template']
        # continuar descendo
        for v in obj.values():
            if isinstance(v, (dict, list)):
                yield from _walk(v)
    elif isinstance(obj, list):
        for it in obj:
            yield from _walk(it)

def extract_templates_json(input_json_path, output_log_path, unique=False, encoding="utf-8"):
    with open(input_json_path, "r", encoding=encoding) as f:
        data = json.load(f)

    out_path = Path(output_log_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    count = 0
    if unique:
        seen = set()
        with open(out_path, "w", encoding=encoding) as fout:
            for tpl in _walk(data):
                tpl = str(tpl).rstrip("\n")
                if tpl not in seen:
                    seen.add(tpl)
                    fout.write(tpl + "\n")
                    count += 1
    else:
        with open(out_path, "w", encoding=encoding) as fout:
            for tpl in _walk(data):
                tpl = str(tpl).rstrip("\n")
                fout.write(tpl + "\n")
                count += 1
    return count

def extract_templates_streaming(input_json_path, output_log_path, unique=False, encoding="utf-8"):
    """
    Usa ijson para extrair qualquer caminho que termine em '.group.item.template'
    sem carregar o JSON completo na memória.
    """
    out_path = Path(output_log_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    count = 0
    seen = set() if unique else None

    with open(input_json_path, "rb") as fin, open(out_path, "w", encoding=encoding) as fout:
        for prefix, event, value in ijson.parse(fin):
            # ex.: "<top_key>.group.item.template"
            if event == "string" and prefix.endswith(".group.item.template"):
                tpl = str(value).rstrip("\n")
                if unique:
                    if tpl in seen:
                        continue
                    seen.add(tpl)
                fout.write(tpl + "\n")
                count += 1
    return count

# --- Célula 3: parâmetros

# Caminho do seu JSON de entrada
INPUT_JSON = "logs/part_221.json"

# Caminho do .log de saída (será criado se não existir)
OUTPUT_LOG = "logs/part_221.log"

# Se True, deduplica (só templates únicos). Se False, mantém repetições e ordem.
UNIQUE = False

# Encoding (ajuste se necessário)
ENCODING = "utf-8"

# --- Célula 4: executar extração

if HAS_IJSON:
    total = extract_templates_streaming(INPUT_JSON, OUTPUT_LOG, unique=UNIQUE, encoding=ENCODING)
else:
    total = extract_templates_json(INPUT_JSON, OUTPUT_LOG, unique=UNIQUE, encoding=ENCODING)

print(f"OK: {total} linha(s) escrita(s) em {OUTPUT_LOG} (unique={UNIQUE}).")


OK: 9569 linha(s) escrita(s) em logs/part_221.log (unique=False).
